In [1]:
import operator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import xgboost as xgb
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_columns', 50)


In [2]:
data_path = "/Users/gaganjain/Downloads/"
score_df = pd.read_csv(data_path+"deliveries.csv")
match_df = pd.read_csv(data_path+"matches.csv")
score_df.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,1,1,SC Ganguly,BB McCullum,P Kumar,0,0,0,1,0,0,0,1,1,NaN,NaN,NaN
1,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,1,2,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,1,3,BB McCullum,SC Ganguly,P Kumar,0,1,0,0,0,0,0,1,1,NaN,NaN,NaN
3,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,1,4,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,1,5,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN


In [3]:
#match_df = match_df.ix[match_df.dl_applied == 0,:]
match_df.shape

(577, 18)

In [4]:
new_score_df = pd.merge(score_df, match_df[['id','season', 'winner', 'result', 'dl_applied', 'team1', 'team2']], left_on='match_id', right_on='id')
new_score_df.player_dismissed.fillna(0, inplace=True)
new_score_df['player_dismissed'].ix[new_score_df['player_dismissed'] != 0] = 1
new_score_df.shape

/Users/gaganjain/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/gaganjain/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


(136598, 28)

In [5]:
train_df = new_score_df.groupby(['match_id', 'inning', 'over', 'team1', 'team2', 'batting_team', 'winner'])[['total_runs', 'player_dismissed']].agg(['sum']).reset_index()
train_df.columns = train_df.columns.get_level_values(0)

In [6]:
train_df.head()

,match_id,inning,over,team1,team2,batting_team,winner,total_runs,player_dismissed
0,1,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,3,0
1,1,1,2,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,18,0
2,1,1,3,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,6,0
3,1,1,4,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,23,0
4,1,1,5,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,10,0


In [7]:
# innings score and wickets #
train_df['innings_wickets'] = train_df.groupby(['match_id', 'inning'])['player_dismissed'].cumsum()
train_df['innings_score'] = train_df.groupby(['match_id', 'inning'])['total_runs'].cumsum()
train_df.head()


,match_id,inning,over,team1,team2,batting_team,winner,total_runs,player_dismissed,innings_wickets,innings_score
0,1,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,3,0,0,3
1,1,1,2,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,18,0,0,21
2,1,1,3,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,6,0,0,27
3,1,1,4,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,23,0,0,50
4,1,1,5,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,10,0,0,60


In [8]:
# Get the target column #
temp_df = train_df.groupby(['match_id', 'inning'])['total_runs'].sum().reset_index()
temp_df = temp_df.ix[temp_df['inning']==1,:]
temp_df['inning'] = 2
temp_df.columns = ['match_id', 'inning', 'score_target']
train_df = train_df.merge(temp_df, how='left', on = ['match_id', 'inning'])
train_df['score_target'].fillna(-1, inplace=True)


/Users/gaganjain/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
train_df.head(n=36)

,match_id,inning,over,team1,team2,batting_team,winner,total_runs,player_dismissed,innings_wickets,innings_score,score_target
0,1,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,3,0,0,3,-1.0
1,1,1,2,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,18,0,0,21,-1.0
2,1,1,3,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,6,0,0,27,-1.0
3,1,1,4,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,23,0,0,50,-1.0
4,1,1,5,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,10,0,0,60,-1.0
5,1,1,6,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,1,1,1,61,-1.0
6,1,1,7,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,7,0,1,68,-1.0
7,1,1,8,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,5,0,1,73,-1.0
8,1,1,9,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,4,0,1,77,-1.0
9,1,1,10,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,10,0,1,87,-1.0


In [10]:
# get the remaining target #
def get_remaining_target(row):
    if row['score_target'] == -1.:
        return -1
    else:
        return row['score_target'] - row['innings_score']

train_df['remaining_target'] = train_df.apply(lambda row: get_remaining_target(row),axis=1)



In [11]:
# get the run rate #
train_df['run_rate'] = train_df['innings_score'] / train_df['over']

# get the remaining run rate #
def get_required_rr(row):
    if row['remaining_target'] == -1:
        return -1.
    elif row['over'] == 20:
        return 99
    else:
        return row['remaining_target'] / (20-row['over'])
    
train_df['required_run_rate'] = train_df.apply(lambda row: get_required_rr(row), axis=1)

def get_rr_diff(row):
    if row['inning'] == 1:
        return -1
    else:
        return row['run_rate'] - row['required_run_rate']
    
train_df['runrate_diff'] = train_df.apply(lambda row: get_rr_diff(row), axis=1)
train_df['is_batting_team'] = (train_df['team1'] == train_df['batting_team']).astype('int')
train_df['target'] = (train_df['team1'] == train_df['winner']).astype('int')



In [12]:
train_df.head(n=41)

,match_id,inning,over,team1,team2,batting_team,winner,total_runs,player_dismissed,innings_wickets,innings_score,score_target,remaining_target,run_rate,required_run_rate,runrate_diff,is_batting_team,target
0,1,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,3,0,0,3,-1.0,-1.0,3.000000,-1.000000,-1.000000,1,1
1,1,1,2,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,18,0,0,21,-1.0,-1.0,10.500000,-1.000000,-1.000000,1,1
2,1,1,3,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,6,0,0,27,-1.0,-1.0,9.000000,-1.000000,-1.000000,1,1
3,1,1,4,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,23,0,0,50,-1.0,-1.0,12.500000,-1.000000,-1.000000,1,1
4,1,1,5,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,10,0,0,60,-1.0,-1.0,12.000000,-1.000000,-1.000000,1,1
5,1,1,6,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,1,1,1,61,-1.0,-1.0,10.166667,-1.000000,-1.000000,1,1
6,1,1,7,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,7,0,1,68,-1.0,-1.0,9.714286,-1.000000,-1.000000,1,1
7,1,1,8,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,5,0,1,73,-1.0,-1.0,9.125000,-1.000000,-1.000000,1,1
8,1,1,9,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,4,0,1,77,-1.0,-1.0,8.555556,-1.000000,-1.000000,1,1
9,1,1,10,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,10,0,1,87,-1.0,-1.0,8.700000,-1.000000,-1.000000,1,1


In [13]:
x_cols = ['inning', 'over', 'total_runs', 'player_dismissed', 'innings_wickets', 'innings_score', 'score_target', 'remaining_target', 'run_rate', 'required_run_rate', 'runrate_diff', 'is_batting_team']


In [32]:

print(train_df.match_id.unique().size)
total_matches = train_df.match_id.unique()

#print(total_matches)

test_data = total_matches[:200]
train_data = total_matches[200:]
#print(test_data)

match_winner = []

#val_df = train_df.ix[train_df.match_id == 2,:]
#print((val_df))


prev_index = 0
test_index = []
for match in test_data:

    val_df = train_df.ix[train_df.match_id == match,:]
    index = val_df.shape[0]
    prev_index = index + prev_index
    winner = val_df['target'][prev_index-1]
    test_index.append(prev_index-1)
    match_winner.append(winner)

print(match_winner)
print(len(test_index))
#target_df = train_df.ix[train_df.match_id in range(20),:]
#print(target_df)
#index = val_df.shape[0]
#print(val_df['target'][index-1])

574
[1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1]
200


In [27]:
# let us take all the matches but for the final as development sample and final as val sample #
"""val_df = train_df.ix[train_df.match_id == 1,:]
index = val_df.shape[0]
print(val_df['target'][index-1])"""

#target_y = train_df.ix[train_df.match_id == 1,:] 

sample_df_final = pd.DataFrame()
for match in range(1, 3):
    sample_df = train_df.ix[train_df.match_id == match,:]
    sample_df_final = sample_df_final.append(sample_df)
    print(sample_df_final.shape)
    




val_df_final = pd.DataFrame()
for match in test_data:
    val_df = train_df.ix[train_df.match_id == match,:]
    val_df_final = val_df_final.append(val_df)
    
#print(val_df_final)

dev_df_final = pd.DataFrame()
for match in train_data:
    dev_df = train_df.ix[train_df.match_id == match,:]
    dev_df_final = dev_df_final.append(dev_df)
    
#dev_df_final

(36, 18)
(76, 18)


In [138]:
#dev_df.head(n=120)
#val_df.loc[val_df['match_id'] == 1]

In [38]:
# create the input and target variables #
dev_X = np.array(dev_df_final[x_cols[:]])
dev_y = np.array(dev_df_final['target'])
val_X = np.array(val_df_final[x_cols[:]])[:-1,:]
val_y = np.array(val_df_final['target'])[:-1]
#val_y  = np.asarray(match_winner)
print(dev_X.shape, dev_y.shape)
print(val_X.shape, val_y.shape)
print(dev_X[0:1])
#dev_y[0:40]

((14407, 12), (14407,))
((7676, 12), (7676,))
[[ 1.  1.  6.  0.  0.  6. -1. -1.  6. -1. -1.  1.]]


In [39]:
#dev_y[0:500]
val_y.size

7676

In [40]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(dev_X, dev_y)

GaussianNB(priors=None)

In [42]:
val_Y_Prediction = clf.predict(val_X)

val_y_array = []
for index in test_index:
    val_y_array.append(val_Y_Prediction[index-1])
    
val_y_np = np.asarray(val_y_array)

val_y_np.shape
#print(val_Y_Prediction.shape)


(200,)

In [144]:

val_df = train_df.ix[train_df.match_id == 206,:]
val_df

"""prev_index = 0
index = 0
val_y_final = []

for match in test_data:
    val_df = train_df.ix[train_df.match_id == match,:]
    index = val_df.shape[0]
    #print(index)
    prev_index = index + prev_index
    print(prev_index)
    #val_y_final.append(val_Y_Prediction[prev_index - 1])"""

#print(val_y_final.size)
#dev_y.size

'prev_index = 0\nindex = 0\nval_y_final = []\n\nfor match in test_data:\n    val_df = train_df.ix[train_df.match_id == match,:]\n    index = val_df.shape[0]\n    #print(index)\n    prev_index = index + prev_index\n    print(prev_index)\n    #val_y_final.append(val_Y_Prediction[prev_index - 1])'

In [131]:
val_df = train_df.ix[train_df.match_id == 206,:]
val_df

,match_id,inning,over,team1,team2,batting_team,winner,total_runs,player_dismissed,innings_wickets,innings_score,score_target,remaining_target,run_rate,required_run_rate,runrate_diff,is_batting_team,target
7710,206,1,1,Deccan Chargers,Kochi Tuskers Kerala,Deccan Chargers,Deccan Chargers,3,1,1,3,-1.0,-1.0,3.000000,-1.000000,-1.000000,1,1
7711,206,1,2,Deccan Chargers,Kochi Tuskers Kerala,Deccan Chargers,Deccan Chargers,1,0,1,4,-1.0,-1.0,2.000000,-1.000000,-1.000000,1,1
7712,206,1,3,Deccan Chargers,Kochi Tuskers Kerala,Deccan Chargers,Deccan Chargers,6,1,2,10,-1.0,-1.0,3.333333,-1.000000,-1.000000,1,1
7713,206,1,4,Deccan Chargers,Kochi Tuskers Kerala,Deccan Chargers,Deccan Chargers,7,0,2,17,-1.0,-1.0,4.250000,-1.000000,-1.000000,1,1
7714,206,1,5,Deccan Chargers,Kochi Tuskers Kerala,Deccan Chargers,Deccan Chargers,2,0,2,19,-1.0,-1.0,3.800000,-1.000000,-1.000000,1,1
7715,206,1,6,Deccan Chargers,Kochi Tuskers Kerala,Deccan Chargers,Deccan Chargers,1,1,3,20,-1.0,-1.0,3.333333,-1.000000,-1.000000,1,1
7716,206,1,7,Deccan Chargers,Kochi Tuskers Kerala,Deccan Chargers,Deccan Chargers,0,0,3,20,-1.0,-1.0,2.857143,-1.000000,-1.000000,1,1
7717,206,1,8,Deccan Chargers,Kochi Tuskers Kerala,Deccan Chargers,Deccan Chargers,9,0,3,29,-1.0,-1.0,3.625000,-1.000000,-1.000000,1,1
7718,206,1,9,Deccan Chargers,Kochi Tuskers Kerala,Deccan Chargers,Deccan Chargers,4,0,3,33,-1.0,-1.0,3.666667,-1.000000,-1.000000,1,1
7719,206,1,10,Deccan Chargers,Kochi Tuskers Kerala,Deccan Chargers,Deccan Chargers,4,0,3,37,-1.0,-1.0,3.700000,-1.000000,-1.000000,1,1


In [43]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(val_y_np, np.asarray(match_winner))

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.90


In [92]:
import yaml
path = '/Users/gaganjain/Downloads/ipl/'
with open(path+'1082591.yaml', 'r') as f:
    doc = yaml.load(f)
    
print type(doc['info']['teams'])
doc

<type 'list'>


{'info': {'city': 'Hyderabad',
  'competition': 'IPL',
  'dates': [datetime.date(2017, 4, 5)],
  'gender': 'male',
  'match_type': 'T20',
  'outcome': {'by': {'runs': 35}, 'winner': 'Sunrisers Hyderabad'},
  'overs': 20,
  'player_of_match': ['Yuvraj Singh'],
  'teams': ['Sunrisers Hyderabad', 'Royal Challengers Bangalore'],
  'toss': {'decision': 'field', 'winner': 'Royal Challengers Bangalore'},
  'umpires': ['AY Dandekar', 'NJ Llong'],
  'venue': 'Rajiv Gandhi International Stadium, Uppal'},
 'innings': [{'1st innings': {'deliveries': [{0.1: {'batsman': 'DA Warner',
       'bowler': 'TS Mills',
       'non_striker': 'S Dhawan',
       'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
     {0.2: {'batsman': 'DA Warner',
       'bowler': 'TS Mills',
       'non_striker': 'S Dhawan',
       'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
     {0.3: {'batsman': 'DA Warner',
       'bowler': 'TS Mills',
       'non_striker': 'S Dhawan',
       'runs': {'batsman': 4, 'extras': 0, 'tot

In [91]:
innings_map = {'1st innings' : 1, '2nd innings': 2, ''}
#extras_map = {'wides'}
batting_team = []
#bowling_team = []

"""batting_team.append(doc['innings'][0]['1st innings']['team'])
bowling_team.append(doc['innings'][1]['2nd innings']['team'])
batting_team.append(bowling_team[0])
bowling_team.append(batting_team[0])"""




for innings in doc['innings']:
    #print innings
    excel_innings_row = []
    for key, value in innings.items() :
        #print key
        teams = doc['info']['teams']
        batting_team = []
        current_batting_team = innings[key]['team']
        batting_team.append(current_batting_team)
        current_bowling_team = (list(set(teams) - set(batting_team)))[0]
        current_innings = innings_map[key]
        excel_innings_row.append(current_innings)
        excel_innings_row.append(current_batting_team)
        excel_innings_row.append(current_bowling_team)
        
        for ball in value['deliveries']:
            excel_over_row = []
            for over, over_details in ball.items() :
                #print 'Over: ', str(over)[0]
                #print 'Ball: ', str(over)[2]
                
                """temp = int(over)
                print 'Temp:',temp
                current_over = temp + 1
                current_ball = (int)((over - temp) * 10)
                print 'After converting:', current_over, current_ball"""
                
                current_over = (int)(over + 1)
                excel_over_row.append(current_over)
                current_ball = str(over)[str(over).index('.')+1]
                excel_over_row.append(current_ball)
                #print over_details
                current_batsman = over_details['batsman']
                excel_over_row.append(current_batsman)
                
                current_nonstriker = over_details['non_striker']
                excel_over_row.append(current_nonstriker)
                
                current_bowler = over_details['bowler']
                excel_over_row.append(current_bowler)
                
                excel_over_row.append(0)

                
                current_total_runs = over_details['runs']['total']
                current_extra_runs = over_details['runs']['extras']
                current_batsman_runs = over_details['runs']['batsman']
                
                current_wide_runs = current_bye_runs = current_legbye_runs = current_noball_runs = 0
                
                if 'extras' in over_details:
                    extra_type = over_details['extras'].keys()[0]
                    if extra_type == 'wides':
                        current_wide_runs = current_extra_runs
                    elif extra_type == 'legbyes':
                        current_legbye_runs = current_extra_runs
                    elif extra_type == 'noballs':
                        current_noball_runs = current_extra_runs
                    elif extra_type == 'bye':
                        current_bye_runs = current_extra_runs
                        
                excel_over_row.append(current_wide_runs)
                excel_over_row.append(current_bye_runs)
                excel_over_row.append(current_legbye_runs)
                excel_over_row.append(current_noball_runs)
                excel_over_row.append(0)
                excel_over_row.append(current_batsman_runs)
                excel_over_row.append(current_extra_runs)
                excel_over_row.append(current_total_runs)
                        
                current_player_dismissed = current_dismissal_kind = current_fielder  = ''
                
                if 'wicket' in over_details:
                    #print over_details['wicket']
                    current_dismissal_kind = over_details['wicket']['kind']
                    current_player_dismissed = over_details['wicket']['player_out']
                    if 'fielders' in over_details['wicket']:
                        current_fielder = over_details['wicket']['fielders'][0]
                        
                excel_over_row.append(current_player_dismissed)
                excel_over_row.append(current_dismissal_kind)
                excel_over_row.append(current_fielder)
                print excel_innings_row + excel_over_row
        
                
            


[1, 'Sunrisers Hyderabad', 'Royal Challengers Bangalore', 1, '1', 'DA Warner', 'S Dhawan', 'TS Mills', 0, 0, 0, 0, 0, 0, 0, 0, 0, '', '', '']
[1, 'Sunrisers Hyderabad', 'Royal Challengers Bangalore', 1, '2', 'DA Warner', 'S Dhawan', 'TS Mills', 0, 0, 0, 0, 0, 0, 0, 0, 0, '', '', '']
[1, 'Sunrisers Hyderabad', 'Royal Challengers Bangalore', 1, '3', 'DA Warner', 'S Dhawan', 'TS Mills', 0, 0, 0, 0, 0, 0, 4, 0, 4, '', '', '']
[1, 'Sunrisers Hyderabad', 'Royal Challengers Bangalore', 1, '4', 'DA Warner', 'S Dhawan', 'TS Mills', 0, 0, 0, 0, 0, 0, 0, 0, 0, '', '', '']
[1, 'Sunrisers Hyderabad', 'Royal Challengers Bangalore', 1, '5', 'DA Warner', 'S Dhawan', 'TS Mills', 0, 2, 0, 0, 0, 0, 0, 2, 2, '', '', '']
[1, 'Sunrisers Hyderabad', 'Royal Challengers Bangalore', 1, '6', 'S Dhawan', 'DA Warner', 'TS Mills', 0, 0, 0, 0, 0, 0, 0, 0, 0, '', '', '']
[1, 'Sunrisers Hyderabad', 'Royal Challengers Bangalore', 1, '7', 'S Dhawan', 'DA Warner', 'TS Mills', 0, 0, 0, 1, 0, 0, 0, 1, 1, '', '', '']
[1, 'S

In [76]:
la = ['a', 'b']
lb = ['b']
print list(set(la) - set(lb))

['a']


In [77]:
la = ['a', 'b']
lb = ['b']
print la + lb

['a', 'b', 'b']


In [80]:
a = b = ''
print a